In [10]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import shutil
from tqdm import tqdm

sys.path.append("..")
from exposure import addExposure

In [32]:
# 在这里设置运行时的参数
normal_path = "normal/"
output_path = "output/"
ratio = 2 

if os.path.exists(output_path):
    shutil.rmtree(output_path)

os.mkdir(output_path) 

In [21]:
# 检测文件
all_files = os.listdir(normal_path)
all_files = sorted(all_files)
data_name = []
for name in all_files:
    prefix, suffix = name.split('.')
    if suffix not in ["jpg", "png"]:
        continue
    if (prefix + ".txt") not in all_files:
        continue
    data_name.append([name, prefix + ".txt"])

In [33]:

for name in tqdm(data_name):
    img_name, label_name = name
    img = cv2.imread(os.path.join(normal_path, img_name))
    label = np.loadtxt(os.path.join(normal_path, label_name), ndmin=2)
    
    # 随机过曝
    height, width, _ = img.shape
    for i in range(ratio):
        copy_img = img.copy()
        bx = label[np.random.randint(0, len(label))]
        category, x, y, w, h = bx
        x1, y1, x2, y2 = int(width * (x - w / 2)), int(height * (y - h / 2)), \
                    int(width * (x + w / 2)), int(height * (y + h / 2))
        center = (np.random.randint(x1, x2), np.random.randint(y1, y2))
        radius = np.random.randint(int(min(y2 - y1, x2 - x1) // 2), int(min(y2 - y1, x2 - x1) * 1.6))
        strength = np.random.randint(255, 400)
        copy_img = addExposure(copy_img, center, radius, strength)

        # 保存结果
        prefix, _ = img_name.split('.')
        new_img_name = prefix + "_{:0>3d}.jpg".format(i + 1)
        new_label_name = prefix + "_{:0>3d}.txt".format(i + 1)
        cv2.imwrite(os.path.join(output_path, new_img_name), copy_img)
        np.savetxt(os.path.join(output_path, new_label_name), label)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:06<00:00,  6.18it/s]
